### Imports

In [ ]:
!pip install transformers
!pip install tweetnlp

In [ ]:
import nltk
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import seaborn as sns
import matplotlib.pyplot as plt
import tweetnlp
import numpy as np

from nltk.corpus import words, stopwords, wordnet
from google.colab import drive
from wordcloud import WordCloud
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from tensorflow.python.client import device_lib
from scipy.special import softmax
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from IPython.display import display

nltk.download('vader_lexicon')
nltk.download('all')
pd.options.mode.chained_assignment = None
device_lib.list_local_devices()

In [ ]:
# mount drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


### Functions and Algorithms

In [ ]:
sid = SentimentIntensityAnalyzer()
modelTweet = tweetnlp.load_model('sentiment')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def frequentWordsWordCloud(tweets):
  word_cloud = WordCloud(width = 1000,
                        height = 800,
                        colormap = 'Blues', 
                        margin = 0,
                        max_words = 200,  
                        min_word_length = 4,
                        max_font_size = 120, 
                        min_font_size = 15,  
                        background_color = "white").generate(" ".join(tweets))

  plt.figure(figsize = (10, 15))
  plt.imshow(word_cloud, interpolation = "gaussian")
  plt.axis("off")
  plt.show()

In [ ]:
def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis = 0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    x, y = map(list, zip(*words_freq))
    return x[:10]

In [ ]:
def vectorizeTweets(doc_term_matrix, n_topics):
  # SVD represent documents and terms in vectors 
  svd_model = TruncatedSVD(n_components=n_topics, algorithm='randomized', n_iter=100, random_state=122)
  svd_model.fit(doc_term_matrix)
  return svd_model

In [ ]:
def lsaAlgo(tweets, max_f, n_topics):
  vectorizer = TfidfVectorizer(stop_words='english', max_features= max_f, max_df = 0.5, smooth_idf=True)
  # document-term matrix
  X = vectorizer.fit_transform(tweets.values.astype('U')) # unicode string
  print("Doc-term matrix shape: ", X.shape)

  # svd vectorization
  svd_model = vectorizeTweets(X, n_topics)
  terms = vectorizer.get_feature_names()
  toptopics=[]
  for i, comp in enumerate(svd_model.components_):
      terms_comp = zip(terms, comp)
      sorted_terms = sorted(terms_comp, key=lambda x:x[1], reverse=True)[:7]
      x, y = map(list, zip(*sorted_terms))
      toptopics.append(' '.join(x))
      # print("Topic "+str(i)+": ", end = ' ')
      #     print(t[0], end = ' ')
      # print()
  return toptopics

In [ ]:
def compute_vader_scores(df):
    df['polarity'] = df['text'].apply(lambda row: sid.polarity_scores(row))
    df['sentiment_vader']= df['polarity'].apply(lambda polarity : polarity['compound'])
    df.drop(columns=['polarity'], inplace=True)
    return df

In [ ]:
def compute_textBlob_scores(df):
  df['sentiment_textBlob'] = df['text'].apply(lambda row: TextBlob(row).sentiment.polarity)
  return df

In [ ]:
def computeTweetSentiment(df):
  df['sentiment_tweet'] = df['text'].apply(lambda row: modelTweet.sentiment(row, return_probability=True))
  df['sentiment_tweet'] = df['sentiment_tweet'].apply(lambda row : 0 if row['label'] == 'neutral' else (-row['probability']['negative'] if row['label'] =='negative' else row['probability']['positive']))
  return df

In [ ]:
def getForAll(df, type):
  sentiment_vadertweets_= df.query('sentiment_vader' + type)

  sentiment_vadertweets_unigram = get_top_ngram(sentiment_vadertweets_.text, 1)
  sentiment_vadertweets_bigram = get_top_ngram(sentiment_vadertweets_.text, 2)
  sentiment_vadertweets_trigram = get_top_ngram(sentiment_vadertweets_.text, 3)
  sentiment_vadertweets_topics = lsaAlgo(sentiment_vadertweets_.text, 1000, 20)

  textBlobtweets_= df.query('sentiment_textBlob'+ type)

  textBlobtweets_unigram = get_top_ngram(textBlobtweets_.text, 1)
  textBlobtweets_bigram = get_top_ngram(textBlobtweets_.text, 2)
  textBlobtweets_trigram = get_top_ngram(textBlobtweets_.text, 3)
  textBlobtweets_topics = lsaAlgo(textBlobtweets_.text, 1000, 20)

  tweetNLPtweets_= df.query('sentiment_tweet'+ type)

  tweetNLPtweets_unigram = get_top_ngram(tweetNLPtweets_.text, 1)
  tweetNLPtweets_bigram = get_top_ngram(tweetNLPtweets_.text, 2)
  tweetNLPtweets_trigram = get_top_ngram(tweetNLPtweets_.text, 3)
  tweetNLPtweets_topics = lsaAlgo(tweetNLPtweets_.text, 1000, 20)

  print("Comparative Analysis for sentiment" + type + " Tweets")
  columns=['Vader','TextBlob','TweetNLP Roberta']

  print("\n-- Unigrams -- ")
  tweets_unigram= pd.DataFrame(list(zip(sentiment_vadertweets_unigram,textBlobtweets_unigram,tweetNLPtweets_unigram)),columns=columns)
  display(tweets_unigram)

  print("\n-- Bigrams -- ")
  tweets_bigram=pd.DataFrame(list(zip(sentiment_vadertweets_bigram,textBlobtweets_bigram,tweetNLPtweets_bigram)),columns=columns)
  display(tweets_bigram)

  print("\n-- Trigrams --")
  tweets_trigram = pd.DataFrame(list(zip(sentiment_vadertweets_trigram,textBlobtweets_trigram,tweetNLPtweets_trigram)),columns=columns)
  display(tweets_trigram)

  print("\n\n-- Topics --")
  tweet_topics=pd.DataFrame(list(zip(sentiment_vadertweets_topics,textBlobtweets_topics,tweetNLPtweets_topics)),columns=columns)
  display(tweet_topics)

In [ ]:
def topicsByTypes(df,label):

  pos_tweets_= df.query(label +'>0')

  pos_tweets_unigram = get_top_ngram(pos_tweets_.text, 1)
  pos_tweets_bigram = get_top_ngram(pos_tweets_.text, 2)
  pos_tweets_trigram = get_top_ngram(pos_tweets_.text, 3)
  pos_tweets_topics = lsaAlgo(pos_tweets_.text, 1000, 20)

  neg_tweets_ = df.query(label +'<0')

  neg_tweets_unigram = get_top_ngram(neg_tweets_.text, 1)
  neg_tweets_bigram = get_top_ngram(neg_tweets_.text, 2)
  neg_tweets_trigram = get_top_ngram(neg_tweets_.text, 3)
  neg_tweets_topics = lsaAlgo(neg_tweets_.text, 1000, 20)

  neu_tweets_ = df.query(label +'==0')
  
  neu_tweets_unigram = get_top_ngram(neu_tweets_.text, 1)
  neu_tweets_bigram = get_top_ngram(neu_tweets_.text, 2)
  neu_tweets_trigram = get_top_ngram(neu_tweets_.text, 3)
  neu_tweets_topics = lsaAlgo(neu_tweets_.text, 1000, 20)

  print("Top Unigrams for all 3 types of tweets: ")
  columns=['Postive Tweets','Negative Tweets','Neutral Tweet']

  print("\n-- Unigrams -- ")
  tweets_unigram=pd.DataFrame(list(zip(pos_tweets_unigram, neg_tweets_unigram, neu_tweets_unigram)), columns=columns)
  display(tweets_unigram)

  print("\n-- Bigrams --")
  tweets_bigram=pd.DataFrame(list(zip(pos_tweets_bigram, neg_tweets_bigram, neu_tweets_bigram)), columns=columns)
  display(tweets_bigram)

  print("\n-- Trigrams --")
  tweets_trigram = pd.DataFrame(list(zip(pos_tweets_trigram,neg_tweets_trigram,neu_tweets_trigram)), columns=columns)
  display(tweets_trigram)

  print("\n\n-- Topics --")
  tweet_topics = pd.DataFrame(list(zip(pos_tweets_topics, neg_tweets_topics, neu_tweets_topics)), columns=columns)
  display(tweet_topics)

##Apple

In [ ]:
apple_tweets = pd.read_csv('/content/gdrive/MyDrive/ai4se/pp_files/apple.csv')
apple_tweets = apple_tweets.dropna()
apple_tweets.head()

,Unnamed: 0,text
1,402,tried resetting settings restarting phone
2,404,looks like
3,415,happening
4,417,tf wrong keyboard
5,419,call centres closed night


In [ ]:
# vader
apple_tweets = compute_vader_scores(apple_tweets)
# text blob
apple_tweets = compute_textBlob_scores(apple_tweets)

In [ ]:
# tweet nlp
apple_tweets = computeTweetSentiment(apple_tweets)

In [ ]:
apple_tweets.head()

,Unnamed: 0,Unnamed: 0.1,text,sentiment_vader,sentiment_textBlob,sentiment_tweet
0,1,402,tried resetting settings restarting phone,0.0000,0.0,0.000000
1,2,404,looks like,0.3612,0.0,0.000000
2,3,415,happening,0.0000,0.0,0.000000
3,4,417,tf wrong keyboard,-0.4767,-0.5,-0.787276
4,5,419,call centres closed night,0.0000,-0.1,0.000000


In [ ]:
apple_tweets.to_csv('/content/apple_with_sentiments.csv')

In [ ]:
# load if already saved because already processed.
apple_tweets = pd.read_csv('/content/apple_with_sentiments.csv')

In [ ]:
# positive sentiments
getForAll(apple_tweets,'>0')

Doc-term matrix shape:  (18520, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (15762, 1000)
Doc-term matrix shape:  (4373, 1000)
Comparative Analysis for sentiment>0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,iphone,iphone,thanks
1,help,phone,iphone
2,phone,new,yes
3,please,update,thank
4,ios,ios,ios
5,update,thanks,update
6,thanks,battery,phone
7,apple,apple,apple
8,fix,fix,new
9,battery,latest,battery



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,please help,new update,iphone plus
1,please fix,ios update,ios update
2,ios update,new ios,iphone 6s
3,iphone 6s,new iphone,battery life
4,need help,high sierra,new iphone
5,battery life,latest update,thank much
6,new update,battery life,new ios
7,iphone plus,latest ios,new update
8,new iphone,iphone 6s,please help
9,apple music,new phone,lag lag



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,new ios update,new ios update,lag lag lag
1,latest ios update,latest ios update,iphone 6s plus
2,ever since updated,macos high sierra,new ios update
3,iphone 6s plus,new iphone update,latest ios update
4,dear please fix,brand new iphone,new iphone update
5,please fix ios,since new update,thanks getting back
6,macos high sierra,new update phone,brand new iphone
7,since ios update,new software update,ios iphone plus
8,since updated ios,iphone 6s plus,worked thank much
9,since last update,lag lag lag,apple watch series




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,help yes iphone thanks phone ios update,update thanks new ios iphone phone battery,yes thank iphone thanks ios plus updated
1,yes restarted times tried happens version started,thanks worked reply fixed ok response great,thank thanks worked ios fixed iphone update
2,thanks ios update fix thank phone iphone,latest ios battery iphone life version fast,thanks iphone ios worked update phone help
3,thank worked fix response fixed appreciate ios,update latest phone version thanks software ru...,iphone plus ios update new 6s battery
4,thanks thank help worked yes fixed reply,battery new update life fast draining drains,plus iphone thanks 6s thank 7s yes
5,fix pls glitch issue help bug dear,iphone new update latest plus thanks 6s,worked think fine plus tried finally perfectly
6,iphone ios battery 6s plus life draining,fixed update new thank issue problem iphone,apple help working phone got plus new
7,apple iphone music like support need app,ios new phone fixed updated iphone bug,update phone new plus love work cool
8,like battery phone updated ios draining looks,ios fix sierra high new macbook app,working help phone works fine updated got
9,ios apple app music updated support store,fix iphone screen phone shit glitch right,fixed working plus update ios apple fine


In [ ]:
# negative sentiments
getForAll(apple_tweets,'<0')

Doc-term matrix shape:  (19209, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (15101, 1000)
Doc-term matrix shape:  (28861, 1000)
Comparative Analysis for sentiment<0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,phone,phone,phone
1,iphone,iphone,iphone
2,ios,ios,update
3,update,update,ios
4,fix,fix,fix
5,battery,battery,battery
6,new,apple,new
7,apple,new,apple
8,shit,shit,please
9,problem,wtf,help



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,battery life,battery life,ios update
1,ios update,ios update,battery life
2,new update,new update,please fix
3,new ios,please fix,new update
4,please fix,fix shit,new ios
5,iphone 6s,new ios,iphone 6s
6,fix shit,iphone 6s,ever since
7,new iphone,ever since,please help
8,ever since,new iphone,update phone
9,every time,iphone plus,new iphone



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,get shit together,get shit together,new ios update
1,new ios update,new ios update,ever since updated
2,ever since updated,ever since updated,get shit together
3,phone keeps freezing,question mark box,latest ios update
4,latest ios update,phone keeps freezing,phone keeps freezing
5,question mark box,since updated ios,since updated ios
6,new iphone update,since ios update,since ios update
7,since updated ios,iphone 6s plus,question mark box
8,since ios update,new iphone update,new iphone update
9,iphone 6s plus,worst update ever,since last update




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,phone update ios iphone fix battery shit,phone update ios fix iphone shit battery,phone update ios iphone fix battery new
1,shit fix wtf question mark glitch fucking,shit fix wtf question mark fucking box,fix shit glitch question mark fucking damn
2,wtf phone going fuck wrong like yo,wtf phone going wrong like went apple,phone shit update wtf new keeps fuck
3,problem wtf iphone battery fix ios life,phone fuck update fucking new fucked slow,battery update phone life ios new draining
4,problem phone apple fuck im freezing app,battery update life phone sucks ios fucked,update new iphone shit latest fix software
5,iphone apple shit screen plus working 6s,fuck battery phone life updated ios fucking,shit iphone battery wtf life fuck like
6,update new fix iphone wtf latest software,fix fucking apple annoying glitch stupid phone,ios shit iphone updated freezing keeps slow
7,shit battery problem fuck iphone life phone,fuck update fucking iphone fix new glitch,iphone fix phone keeps freezing ios11 glitch
8,ios freezing iphone shit keeps problem apps,ios slow fuck fix updated annoying fucking,help shit update need keeps pls wont
9,apple ios shit update music problem new,ios apple fuck update worst app music,keeps freezing crashing apps app update ios11


In [ ]:
# neutral sentiment
getForAll(apple_tweets,'==0')

Doc-term matrix shape:  (18995, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (25861, 1000)
Doc-term matrix shape:  (23490, 1000)
Comparative Analysis for sentiment==0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,iphone,iphone,iphone
1,ios,ios,ios
2,phone,phone,phone
3,update,fix,apple
4,fix,update,update
5,apple,apple,help
6,battery,help,fix
7,new,please,new
8,app,battery,please
9,still,app,app



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,iphone 6s,iphone 6s,iphone 6s
1,iphone plus,iphone plus,iphone plus
2,ios update,please fix,apple music
3,apple music,please help,please help
4,new ios,apple music,apple watch
5,new update,ios update,ios update
6,iphone ios,every time,high sierra
7,battery life,iphone ios,iphone ios
8,apple watch,battery life,new iphone
9,high sierra,apple watch,apple id



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,iphone 6s plus,iphone 6s plus,iphone 6s plus
1,iphone 6s ios,iphone 6s ios,iphone 6s ios
2,new ios update,ever since updated,battery health iphone
3,iphone plus ios,iphone plus ios,apple watch series
4,latest ios update,battery health iphone,iphone plus ios
5,battery health iphone,since updated ios,macos high sierra
6,ever since updated,since ios update,apple pay cash
7,apple watch series,question mark box,check battery health
8,since updated ios,apple watch series,overall battery health
9,question mark box,since last update,new ios update




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,ios iphone 6s plus update phone fix,ios iphone fix 6s phone update help,ios iphone 6s update plus phone help
1,iphone 6s plus 5s check se health,fix help phone update need apple glitch,help fix need iphone phone apple update
2,fix phone update iphone new issue battery,yes help tried updated times iphone thank,fix ios phone update issue help going
3,phone update new updated battery apple latest,help iphone need apple hi 6s plus,fix iphone phone 6s update issue battery
4,1103 phone iphone ios updated charging started,ios help phone updated yes update need,phone update apple new updated app time
5,update battery new latest apple issue draining,phone update updated apple keeps app apps,update latest new issue battery software recent
6,6s plus update 1103 battery latest version,thank worked apple got problem issue response,1103 update latest new help iphone problem
7,apple music app 6s watch issue store,apple update music app watch problem store,apple music app watch store id hi
8,plus update version apple app music new,update battery 6s help life recent draining,6s apple plus music 1103 app yes
9,battery issue life check draining health way,6s plus apple battery music app watch,issue yes battery updated problem check dm


In [ ]:
topicsByTypes(apple_tweets, 'sentiment_vader')

Doc-term matrix shape:  (18520, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (19209, 1000)
Doc-term matrix shape:  (18995, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,iphone,phone,iphone
1,help,iphone,ios
2,phone,ios,phone
3,please,update,update
4,ios,fix,fix
5,update,battery,apple
6,thanks,new,battery
7,apple,apple,new
8,fix,shit,app
9,battery,problem,still



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,please help,battery life,iphone 6s
1,please fix,ios update,iphone plus
2,ios update,new update,ios update
3,iphone 6s,new ios,apple music
4,need help,please fix,new ios
5,battery life,iphone 6s,new update
6,new update,fix shit,iphone ios
7,iphone plus,new iphone,battery life
8,new iphone,ever since,apple watch
9,apple music,every time,high sierra



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,new ios update,get shit together,iphone 6s plus
1,latest ios update,new ios update,iphone 6s ios
2,ever since updated,ever since updated,new ios update
3,iphone 6s plus,phone keeps freezing,iphone plus ios
4,dear please fix,latest ios update,latest ios update
5,please fix ios,question mark box,battery health iphone
6,macos high sierra,new iphone update,ever since updated
7,since ios update,since updated ios,apple watch series
8,since updated ios,since ios update,since updated ios
9,since last update,iphone 6s plus,question mark box




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,help yes iphone thanks phone ios update,phone update ios iphone fix battery shit,ios iphone 6s plus update phone fix
1,yes restarted times tried happens version started,shit fix wtf question mark glitch fucking,iphone 6s plus 5s check se health
2,thanks ios update fix thank phone iphone,wtf phone going fuck wrong like yo,fix phone update iphone new issue battery
3,thank worked fix response fixed appreciate ios,problem wtf iphone battery fix ios life,phone update new updated battery apple latest
4,thanks thank help worked yes fixed reply,problem phone apple fuck im freezing app,1103 phone iphone ios updated charging started
5,fix pls glitch issue help bug dear,iphone apple shit screen plus working 6s,update battery new latest apple issue draining
6,iphone ios battery 6s plus life draining,update new fix iphone wtf latest software,6s plus update 1103 battery latest version
7,apple iphone music like support need app,shit battery problem fuck iphone life phone,apple music app 6s watch issue store
8,like battery phone updated ios draining looks,ios freezing iphone shit keeps problem apps,plus update version apple app music new
9,ios apple app music updated support store,apple ios shit update music problem new,battery issue life check draining health way


In [ ]:
topicsByTypes(apple_tweets, 'sentiment_tweet')

Doc-term matrix shape:  (4373, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (28861, 1000)
Doc-term matrix shape:  (23490, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks,phone,iphone
1,iphone,iphone,ios
2,yes,update,phone
3,thank,ios,apple
4,ios,fix,update
5,update,battery,help
6,phone,new,fix
7,apple,apple,new
8,new,please,please
9,battery,help,app



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,iphone plus,ios update,iphone 6s
1,ios update,battery life,iphone plus
2,iphone 6s,please fix,apple music
3,battery life,new update,please help
4,new iphone,new ios,apple watch
5,thank much,iphone 6s,ios update
6,new ios,ever since,high sierra
7,new update,please help,iphone ios
8,please help,update phone,new iphone
9,lag lag,new iphone,apple id



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,lag lag lag,new ios update,iphone 6s plus
1,iphone 6s plus,ever since updated,iphone 6s ios
2,new ios update,get shit together,battery health iphone
3,latest ios update,latest ios update,apple watch series
4,new iphone update,phone keeps freezing,iphone plus ios
5,thanks getting back,since updated ios,macos high sierra
6,brand new iphone,since ios update,apple pay cash
7,ios iphone plus,question mark box,check battery health
8,worked thank much,new iphone update,overall battery health
9,apple watch series,since last update,new ios update




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,yes thank iphone thanks ios plus updated,phone update ios iphone fix battery new,ios iphone 6s update plus phone help
1,thank thanks worked ios fixed iphone update,fix shit glitch question mark fucking damn,help fix need iphone phone apple update
2,thanks iphone ios worked update phone help,phone shit update wtf new keeps fuck,fix ios phone update issue help going
3,iphone plus ios update new 6s battery,battery update phone life ios new draining,fix iphone phone 6s update issue battery
4,plus iphone thanks 6s thank 7s yes,update new iphone shit latest fix software,phone update apple new updated app time
5,worked think fine plus tried finally perfectly,shit iphone battery wtf life fuck like,update latest new issue battery software recent
6,apple help working phone got plus new,ios shit iphone updated freezing keeps slow,1103 update latest new help iphone problem
7,update phone new plus love work cool,iphone fix phone keeps freezing ios11 glitch,apple music app watch store id hi
8,working help phone works fine updated got,help shit update need keeps pls wont,6s apple plus music 1103 app yes
9,fixed working plus update ios apple fine,keeps freezing crashing apps app update ios11,issue yes battery updated problem check dm


In [ ]:
topicsByTypes(apple_tweets,'sentiment_textBlob')

Doc-term matrix shape:  (15762, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (15101, 1000)
Doc-term matrix shape:  (25861, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,iphone,phone,iphone
1,phone,iphone,ios
2,new,ios,phone
3,update,update,fix
4,ios,fix,update
5,thanks,battery,apple
6,battery,apple,help
7,apple,new,please
8,fix,shit,battery
9,latest,wtf,app



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,new update,battery life,iphone 6s
1,ios update,ios update,iphone plus
2,new ios,new update,please fix
3,new iphone,please fix,please help
4,high sierra,fix shit,apple music
5,latest update,new ios,ios update
6,battery life,iphone 6s,every time
7,latest ios,ever since,iphone ios
8,iphone 6s,new iphone,battery life
9,new phone,iphone plus,apple watch



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,new ios update,get shit together,iphone 6s plus
1,latest ios update,new ios update,iphone 6s ios
2,macos high sierra,ever since updated,ever since updated
3,new iphone update,question mark box,iphone plus ios
4,brand new iphone,phone keeps freezing,battery health iphone
5,since new update,since updated ios,since updated ios
6,new update phone,since ios update,since ios update
7,new software update,iphone 6s plus,question mark box
8,iphone 6s plus,new iphone update,apple watch series
9,lag lag lag,worst update ever,since last update




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,update thanks new ios iphone phone battery,phone update ios fix iphone shit battery,ios iphone fix 6s phone update help
1,thanks worked reply fixed ok response great,shit fix wtf question mark fucking box,fix help phone update need apple glitch
2,latest ios battery iphone life version fast,wtf phone going wrong like went apple,yes help tried updated times iphone thank
3,update latest phone version thanks software ru...,phone fuck update fucking new fucked slow,help iphone need apple hi 6s plus
4,battery new update life fast draining drains,battery update life phone sucks ios fucked,ios help phone updated yes update need
5,iphone new update latest plus thanks 6s,fuck battery phone life updated ios fucking,phone update updated apple keeps app apps
6,fixed update new thank issue problem iphone,fix fucking apple annoying glitch stupid phone,thank worked apple got problem issue response
7,ios new phone fixed updated iphone bug,fuck update fucking iphone fix new glitch,apple update music app watch problem store
8,ios fix sierra high new macbook app,ios slow fuck fix updated annoying fucking,update battery 6s help life recent draining
9,fix iphone screen phone shit glitch right,ios apple fuck update worst app music,6s plus apple battery music app watch


## Spotify

In [ ]:
spotify_tweets = pd.read_csv('/content/gdrive/MyDrive/ai4se/pp_files/spotify.csv')
spotify_tweets = spotify_tweets.dropna()
spotify_tweets.head()

,Unnamed: 0,text
0,545,iphone recent update spotify
1,553,groove music quits amp redirect spotify w10m a...
2,555,ok thx
3,557,way find nonexplicit songs explicit
4,1275,1065320gac7a8e02


In [ ]:
# vader
spotify_tweets = compute_vader_scores(spotify_tweets)
# text blob
spotify_tweets = compute_textBlob_scores(spotify_tweets)

In [ ]:
# tweet nlp
spotify_tweets = computeTweetSentiment(spotify_tweets)

In [ ]:
spotify_tweets.head()

,Unnamed: 0,text,sentiment_vader,sentiment_textBlob,sentiment_tweet
0,545,iphone recent update spotify,0.0000,0.0,0.000000
1,553,groove music quits amp redirect spotify w10m a...,0.0000,0.0,0.000000
2,555,ok thx,0.5719,0.5,0.559984
3,557,way find nonexplicit songs explicit,0.0000,0.0,0.000000
4,1275,1065320gac7a8e02,0.0000,0.0,0.000000


In [ ]:
spotify_tweets.to_csv('/content/spotify_with_sentiments.csv')

In [ ]:
# load if already saved because already processed.
spotify_tweets = pd.read_csv('/content/spotify_with_sentiments.csv')

In [ ]:
# positive sentiments
getForAll(spotify_tweets,'>0')

Doc-term matrix shape:  (14133, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (9954, 1000)
Doc-term matrix shape:  (4002, 1000)
Comparative Analysis for sentiment>0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,spotify,spotify,thanks
1,help,thanks,thank
2,please,account,spotify
3,account,premium,would
4,thanks,new,please
5,premium,app,love
6,like,im,app
7,app,please,great
8,im,songs,good
9,songs,music,music



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,please help,premium account,thank much
1,need help,spotify premium,would love
2,premium account,please help,customer service
3,spotify premium,spotify account,would great
4,spotify account,new album,thanks help
5,family plan,new account,thanks reply
6,credit card,family plan,bring back
7,would like,still free,spotify premium
8,help please,discover weekly,thanks guys
9,premium family,song available,spotify version



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,account please help,ad free music,thanks getting back
1,need help account,minutes ad free,thanks quick response
2,ad free music,account still free,apple watch app
3,csrf token invalid,still says free,please bring back
4,account still free,latest version spotify,todays top hits
5,please bring back,charged full price,keep good work
6,minutes ad free,todays top hits,would love see
7,still says free,says song available,great customer service
8,apple watch app,day free trial,app available india
9,token invalid please,premium family account,seems working fine




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,thanks help thank account spotify premium need,thanks spotify help account premium hi im,thank thanks worked help okay spotify ok
1,thank help account spotify need premium free,spotify account premium free new available help,thanks help guys spotify reply good ok
2,thank thanks worked okay ok reply response,account premium free help family email charged,yes spotify love iphone app good working
3,help need thanks thank dm sent password,thank ok okay good love fixed really,spotify love iphone good app guys great
4,account premium free family email student charged,available thank free premium account spotify r...,iphone update app ios support version updated
5,spotify version iphone help bring ios android,new album account thank available playlist email,good iphone update support ios app fixed
6,yes app iphone update support premium im,spotify new album latest version iphone ios,help working spotify fine im premium great
7,yes spotify songs account playlist like email,spotify love really ok good playlist like,spotify working worked version fine premium th...
8,premium free help im family student songs,free album premium trial songs music ad,worked guys spotify great awesome ok cool
9,need dm like premium im family check,ok im sure songs works album fine,worked love working ok help fine app


In [ ]:
# negative sentiments
getForAll(spotify_tweets,'<0')

Doc-term matrix shape:  (7060, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (4914, 1000)
Doc-term matrix shape:  (10909, 1000)
Comparative Analysis for sentiment<0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,spotify,spotify,spotify
1,account,account,account
2,premium,im,cant
3,im,app,help
4,app,songs,premium
5,cant,premium,app
6,get,get,im
7,songs,music,please
8,music,help,songs
9,dont,please,get



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,premium account,please help,please help
1,spotify premium,premium account,premium account
2,cant play,went wrong,spotify account
3,spotify account,family plan,spotify premium
4,student discount,web player,doesnt work
5,family plan,something went,cant log
6,keep getting,spotify premium,family plan
7,web player,spotify account,web player
8,family account,every time,need help
9,account hacked,customer service,every time



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,something went wrong,something went wrong,something went wrong
1,keep getting error,oops something went,oops something went
2,oops something went,went wrong please,csrf token invalid
3,get shit together,wrong please try,keep getting error
4,getting error message,get shit together,every time try
5,premium family account,premium family account,cant log account
6,someone hacked account,provider bad day,went wrong please
7,charged full price,spotify premium account,premium family account
8,got charged twice,apple watch app,get shit together
9,spotify web player,discover weekly playlist,wrong please try




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,account spotify premium im charged family app,spotify account im premium help app songs,account spotify help premium im email app
1,account charged premium student email family c...,account family help premium trouble email wrong,account email help premium hacked family changed
2,problem account hacked email family error help,fuck spotify wtf shit account hulu care,spotify premium family student charged im plan
3,problem charged student premium im app month,fuck ill thanks try wrong thank went,spotify songs dont account email want playlist
4,spotify problem fuck pay wtf isnt hulu,wrong song songs playlist artist family album,spotify app log email facebook fix password
5,error family app im getting try message,wtf spotify wrong ill try thanks yo,help play doesnt work song spotify error
6,app music update iphone fix stop account,app wrong iphone update artist song desktop,work doesnt dont account email premium didnt
7,pay premium dont family want cancel stop,spotify wrong song artist hate album went,help dont need charged didnt want student
8,im stop music student offline trying paying,im shit family music premium plan listen,im music dont password trying email doesnt
9,wrong wtf fuck shit song artist album,im song trouble help album wtf logging,family dont plan want email wrong like


In [ ]:
# neutral sentiment
getForAll(spotify_tweets,'==0')

Doc-term matrix shape:  (9728, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (16053, 1000)
Doc-term matrix shape:  (16010, 1000)
Comparative Analysis for sentiment==0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,spotify,spotify,spotify
1,account,account,account
2,premium,premium,premium
3,app,help,please
4,cant,please,app
5,get,app,help
6,im,cant,get
7,version,im,im
8,still,get,songs
9,email,songs,music



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,spotify version,please help,spotify premium
1,spotify premium,spotify premium,premium account
2,premium account,premium account,please help
3,spotify account,need help,need help
4,student discount,spotify account,spotify version
5,im using,spotify version,spotify account
6,web player,student discount,student discount
7,spotify app,family plan,family plan
8,ios spotify,premium family,premium family
9,family plan,email address,im using



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,ios spotify version,apple watch app,apple watch app
1,apple watch app,ios spotify version,ios spotify version
2,spotify version armv7,csrf token invalid,premium family account
3,iphone ios spotify,keep getting error,need help account
4,update app iphone,need help account,spotify version armv7
5,android spotify version,iphone ios spotify,account still free
6,latest version spotify,premium family account,spotify premium account
7,samsung galaxy s7,every time try,iphone ios spotify
8,iphone 6s ios,account please help,latest version spotify
9,spotify version 1065320gac7a8e02,trying update payment,discover weekly playlist




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,spotify version iphone ios app account premium,spotify help account premium need im email,spotify account help premium version need app
1,iphone version ios app 6s android update,thank worked yes dm iphone sent reply,help account need premium family email dm
2,spotify available reputation album isnt india ...,help account need email premium family dm,help need iphone version ios app spotify
3,app available album songs music hey isnt,help iphone app need ios version update,app iphone version ios songs update playlist
4,version android im armv7 using samsung galaxy,app iphone update ios songs im hey,songs playlist album song available music like
5,album available reputation new email songs song,iphone account ios version email update 6s,dm sent check hi email thanks reply
6,email app spotify address facebook changed acc...,premium student im charged family help discount,version im premium android songs playlist student
7,dm sent songs playlist hey im music,dm sent hi check reply guys pls,email account version songs playlist album ios
8,dm sent available reputation working premium isnt,app spotify working watch apple desktop fix,album available reputation new version isnt pr...
9,working email im work isnt album tried,im email version android help using app,android app im email version using music


In [ ]:
topicsByTypes(spotify_tweets, 'sentiment_vader')

Doc-term matrix shape:  (14133, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (7060, 1000)
Doc-term matrix shape:  (9728, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,spotify,spotify,spotify
1,help,account,account
2,please,premium,premium
3,account,im,app
4,thanks,app,cant
5,premium,cant,get
6,like,get,im
7,app,songs,version
8,im,music,still
9,songs,dont,email



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,please help,premium account,spotify version
1,need help,spotify premium,spotify premium
2,premium account,cant play,premium account
3,spotify premium,spotify account,spotify account
4,spotify account,student discount,student discount
5,family plan,family plan,im using
6,credit card,keep getting,web player
7,would like,web player,spotify app
8,help please,family account,ios spotify
9,premium family,account hacked,family plan



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,account please help,something went wrong,ios spotify version
1,need help account,keep getting error,apple watch app
2,ad free music,oops something went,spotify version armv7
3,csrf token invalid,get shit together,iphone ios spotify
4,account still free,getting error message,update app iphone
5,please bring back,premium family account,android spotify version
6,minutes ad free,someone hacked account,latest version spotify
7,still says free,charged full price,samsung galaxy s7
8,apple watch app,got charged twice,iphone 6s ios
9,token invalid please,spotify web player,spotify version 1065320gac7a8e02




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks help thank account spotify premium need,account spotify premium im charged family app,spotify version iphone ios app account premium
1,thank help account spotify need premium free,account charged premium student email family c...,iphone version ios app 6s android update
2,thank thanks worked okay ok reply response,problem account hacked email family error help,spotify available reputation album isnt india ...
3,help need thanks thank dm sent password,problem charged student premium im app month,app available album songs music hey isnt
4,account premium free family email student charged,spotify problem fuck pay wtf isnt hulu,version android im armv7 using samsung galaxy
5,spotify version iphone help bring ios android,error family app im getting try message,album available reputation new email songs song
6,yes app iphone update support premium im,app music update iphone fix stop account,email app spotify address facebook changed acc...
7,yes spotify songs account playlist like email,pay premium dont family want cancel stop,dm sent songs playlist hey im music
8,premium free help im family student songs,im stop music student offline trying paying,dm sent available reputation working premium isnt
9,need dm like premium im family check,wrong wtf fuck shit song artist album,working email im work isnt album tried


In [ ]:
topicsByTypes(spotify_tweets, 'sentiment_tweet')

Doc-term matrix shape:  (4002, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (10909, 1000)
Doc-term matrix shape:  (16010, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks,spotify,spotify
1,thank,account,account
2,spotify,cant,premium
3,would,help,please
4,please,premium,app
5,love,app,help
6,app,im,get
7,great,please,im
8,good,songs,songs
9,music,get,music



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thank much,please help,spotify premium
1,would love,premium account,premium account
2,customer service,spotify account,please help
3,would great,spotify premium,need help
4,thanks help,doesnt work,spotify version
5,thanks reply,cant log,spotify account
6,bring back,family plan,student discount
7,spotify premium,web player,family plan
8,thanks guys,need help,premium family
9,spotify version,every time,im using



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks getting back,something went wrong,apple watch app
1,thanks quick response,oops something went,ios spotify version
2,apple watch app,csrf token invalid,premium family account
3,please bring back,keep getting error,need help account
4,todays top hits,every time try,spotify version armv7
5,keep good work,cant log account,account still free
6,would love see,went wrong please,spotify premium account
7,great customer service,premium family account,iphone ios spotify
8,app available india,get shit together,latest version spotify
9,seems working fine,wrong please try,discover weekly playlist




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thank thanks worked help okay spotify ok,account spotify help premium im email app,spotify account help premium version need app
1,thanks help guys spotify reply good ok,account email help premium hacked family changed,help account need premium family email dm
2,yes spotify love iphone app good working,spotify premium family student charged im plan,help need iphone version ios app spotify
3,spotify love iphone good app guys great,spotify songs dont account email want playlist,app iphone version ios songs update playlist
4,iphone update app ios support version updated,spotify app log email facebook fix password,songs playlist album song available music like
5,good iphone update support ios app fixed,help play doesnt work song spotify error,dm sent check hi email thanks reply
6,help working spotify fine im premium great,work doesnt dont account email premium didnt,version im premium android songs playlist student
7,spotify working worked version fine premium th...,help dont need charged didnt want student,email account version songs playlist album ios
8,worked guys spotify great awesome ok cool,im music dont password trying email doesnt,album available reputation new version isnt pr...
9,worked love working ok help fine app,family dont plan want email wrong like,android app im email version using music


In [ ]:
topicsByTypes(spotify_tweets,'sentiment_textBlob')

Doc-term matrix shape:  (9954, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (4914, 1000)
Doc-term matrix shape:  (16053, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,spotify,spotify,spotify
1,thanks,account,account
2,account,im,premium
3,premium,app,help
4,new,songs,please
5,app,premium,app
6,im,get,cant
7,please,music,im
8,songs,help,get
9,music,please,songs



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,premium account,please help,please help
1,spotify premium,premium account,spotify premium
2,please help,went wrong,premium account
3,spotify account,family plan,need help
4,new album,web player,spotify account
5,new account,something went,spotify version
6,family plan,spotify premium,student discount
7,still free,spotify account,family plan
8,discover weekly,every time,premium family
9,song available,customer service,email address



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,ad free music,something went wrong,apple watch app
1,minutes ad free,oops something went,ios spotify version
2,account still free,went wrong please,csrf token invalid
3,still says free,wrong please try,keep getting error
4,latest version spotify,get shit together,need help account
5,charged full price,premium family account,iphone ios spotify
6,todays top hits,provider bad day,premium family account
7,says song available,spotify premium account,every time try
8,day free trial,apple watch app,account please help
9,premium family account,discover weekly playlist,trying update payment




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks spotify help account premium hi im,spotify account im premium help app songs,spotify help account premium need im email
1,spotify account premium free new available help,account family help premium trouble email wrong,thank worked yes dm iphone sent reply
2,account premium free help family email charged,fuck spotify wtf shit account hulu care,help account need email premium family dm
3,thank ok okay good love fixed really,fuck ill thanks try wrong thank went,help iphone app need ios version update
4,available thank free premium account spotify r...,wrong song songs playlist artist family album,app iphone update ios songs im hey
5,new album account thank available playlist email,wtf spotify wrong ill try thanks yo,iphone account ios version email update 6s
6,spotify new album latest version iphone ios,app wrong iphone update artist song desktop,premium student im charged family help discount
7,spotify love really ok good playlist like,spotify wrong song artist hate album went,dm sent hi check reply guys pls
8,free album premium trial songs music ad,im shit family music premium plan listen,app spotify working watch apple desktop fix
9,ok im sure songs works album fine,im song trouble help album wtf logging,im email version android help using app


## Uber

In [ ]:
uber_tweets = pd.read_csv('/content/gdrive/MyDrive/ai4se/pp_files/uber.csv')
uber_tweets = uber_tweets.dropna()
uber_tweets.head()

,Unnamed: 0,text
0,466,need correct false charges trip lexington ky s...
1,468,driver drove department air travel instead air...
2,472,got threw door today went order sorry dont del...
3,475,phone number im literally stranded wait automa...
4,479,hour inside old altima smells ripped stain se...


In [ ]:
# vader
uber_tweets = compute_vader_scores(uber_tweets)
# text blob
uber_tweets = compute_textBlob_scores(uber_tweets)

In [ ]:
# tweet nlp
uber_tweets = computeTweetSentiment(uber_tweets)

In [ ]:
uber_tweets.head()

,Unnamed: 0,text,sentiment_vader,sentiment_textBlob,sentiment_tweet
0,466,need correct false charges trip lexington ky s...,-0.2023,-0.200000,-0.807215
1,468,driver drove department air travel instead air...,0.0000,0.000000,0.000000
2,472,got threw door today went order sorry dont del...,-0.0772,-0.500000,-0.842147
3,475,phone number im literally stranded wait automa...,-0.2960,-0.333333,-0.938479
4,479,hour inside old altima smells ripped stain se...,0.0000,0.000000,-0.867064


In [ ]:
uber_tweets.to_csv('/content/uber_with_sentiments.csv')

In [ ]:
# load if already saved because already processed.
uber_tweets = pd.read_csv('/content/uber_with_sentiments.csv')

In [ ]:
# positive sentiments
getForAll(uber_tweets,'>0')

Doc-term matrix shape:  (15593, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (10015, 1000)
Doc-term matrix shape:  (1700, 1000)
Comparative Analysis for sentiment>0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,help,uber,thanks
1,uber,driver,uber
2,please,ride,driver
3,driver,get,done
4,account,help,thank
5,get,thanks,help
6,app,app,get
7,ride,account,please
8,email,please,ride
9,support,drivers,would



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,please help,uber driver,customer service
1,need help,please help,would love
2,phone number,customer service,ride pass
3,uber driver,first time,please help
4,customer service,uber eats,thank much
5,credit card,phone number,done thanks
6,would like,promo code,uber eats
7,uber account,last night,promo code
8,uber eats,uber account,sent dm
9,please check,need help,email address



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,left phone uber,left phone uber,thanks quick response
1,please check dm,first time using,thanks sent dm
2,please let know,charged cancellation fee,great customer service
3,customer care number,able book cab,please respond dm
4,want money back,charged full price,thanks prompt response
5,lost phone uber,please let know,get phone back
6,left phone car,driver cancelled trip,would love get
7,friend left phone,able use uber,another ride pass
8,charged cancellation fee,able book ride,promo code received
9,please help get,want full refund,thanks charging ride




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,help uber need account phone driver number,uber thanks driver help ride account app,thanks sent thank dm help email uber
1,help need section disabled invoices trouble error,thanks sent dm email check reply response,thank email sent help appreciate response got
2,thanks dm sent check email thank reply,account help email able number phone new,help uber driver service great good ride
3,number phone email contact account support cus...,ride free able charged code pass promo,sent dm help email response forward asap
4,thanks phone number left car contact lost,uber ride driver account available new phone,help need working app absolutely yes talk
5,account email support thanks card credit address,uber drivers available free help need really,service email customer great good support really
6,account uber dm check credit card disabled,help need really driver able ride trip,email helpful ride working address pass got
7,email uber thank help phone left address,email drivers really need ride sent uber,app working support response code helpful uber
8,support uber customer app team issue help,phone drivers number new account really car,love hope response good reply ride soon
9,need uber support contact email asap driver,drivers account trip able service charged cancel,good driver hope uber response soon need


In [ ]:
# negative sentiments
getForAll(uber_tweets,'<0')

Doc-term matrix shape:  (16245, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (11822, 1000)
Doc-term matrix shape:  (23771, 1000)
Comparative Analysis for sentiment<0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,driver,driver,driver
1,uber,uber,uber
2,charged,service,help
3,ride,get,get
4,get,ride,ride
5,drivers,app,app
6,service,help,account
7,app,drivers,charged
8,trip,account,drivers
9,order,wrong,cant



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,customer service,customer service,customer service
1,uber driver,uber driver,uber driver
2,driver cancelled,please help,please help
3,cancellation fee,uber eats,driver cancelled
4,uber eats,account disabled,cancellation fee
5,got charged,cancellation fee,uber eats
6,last night,last night,last night
7,cancelled trip,phone number,phone number
8,get charged,driver cancelled,uber account
9,charged cancellation,customer support,got charged



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,charged cancellation fee,worst customer service,charged cancellation fee
1,driver cancelled trip,want money back,driver cancelled trip
2,worst customer service,charged cancellation fee,want money back
3,want money back,driver cancelled trip,left phone uber
4,something went wrong,something went wrong,worst customer service
5,lost phone uber,terrible customer service,lost phone uber
6,driver cancelled ride,horrible customer service,driver never showed
7,charged ride never,customer service ever,get money back
8,driver never showed,poor customer service,something went wrong
9,get money back,worst service ever,driver cancelled ride




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,driver uber charged ride cancelled trip cancel,uber driver service customer ride help wrong,uber driver help charged ride account trip
1,charged cancelled trip ride driver cancel fee,service customer worst terrible experience hor...,account help phone uber email number app
2,uber account phone driver lost email help,account disabled help email unable support phone,service customer order worst food drivers app
3,charged account ride order got email help,order wrong food ubereats delivered missing cold,uber service customer driver worst drivers phone
4,order wrong food cancelled driver ubereats del...,uber eats order worst food experience missing,order uber food wrong driver eats ubereats
5,uber cancelled charged order got service eats,account disabled driver service order uber worst,uber account charged got drivers ride order
6,ride drivers uber order cancel time eats,drivers time wtf account long worst disabled,drivers app ride dont im cancel time
7,ride driver phone wrong lost service customer,ride charged help wtf uber unable wrong,drivers account cancel phone disabled trip dont
8,cancelled account ride order hacked pay cancel,worst drivers help wrong experience driver phone,app trip account driver email support use
9,drivers cancelled phone lost got number car,wrong complaint charged drivers worst email trip,help trip drivers charged uber cancelled need


In [ ]:
# neutral sentiment
getForAll(uber_tweets,'==0')

Doc-term matrix shape:  (10106, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (20107, 1000)
Doc-term matrix shape:  (16473, 1000)
Comparative Analysis for sentiment==0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,uber,uber,uber
1,driver,driver,driver
2,email,help,please
3,app,please,email
4,get,get,help
5,ride,email,get
6,account,account,ride
7,still,ride,account
8,sent,app,need
9,need,charged,app



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,uber eats,please help,please help
1,uber driver,uber driver,uber eats
2,ride pass,uber eats,uber driver
3,promo code,need help,need help
4,still waiting,phone number,phone number
5,left phone,left phone,email address
6,sent dm,last night,please check
7,uber app,uber account,left phone
8,email address,cancellation fee,sent dm
9,last night,phone uber,ride pass



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,left phone uber,left phone uber,left phone uber
1,email address email,charged cancellation fee,please check dm
2,uber last night,lost phone uber,email id email
3,still waiting response,uber last night,email address email
4,email id email,want money back,left phone car
5,uber eats deliver,left phone car,uber last night
6,im still waiting,driver cancelled trip,please let know
7,mail id email,please check dm,uber eats deliver
8,havent heard anything,get money back,change phone number
9,left phone car,customer care number,customer care number




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,email sent dm address account id uber,email help uber driver account need phone,email help sent dm uber account need
1,sent dm check response uber reply message,email address id mail registered mob replied,help dm sent uber need driver account
2,uber driver account app ride eats phone,dm sent check reply response thank message,dm sent check reply message respond response
3,dm check reply need whats follow send,help need account asap email pls issue,help need email dm asap urgent pls
4,response waiting reply received hours ive app,uber phone account number eats contact left,sent message response note help thank ive
5,account disabled need app ride working delete,sent response message note app thank ive,number phone contact account need customer left
6,uber eats response account disabled deliver dm,phone driver number left contact car lost,ride charged driver trip need response pass
7,app ride order eats uber working code,account number phone need contact app support,account need ride card use credit app
8,waiting reply order im eats hour minutes,app response waiting order support issue reply,response waiting reply app issue support received
9,ride need pass uber reply waiting charge,response account waiting driver trip help reply,need uber asap contact eats drivers speak


In [ ]:
topicsByTypes(uber_tweets, 'sentiment_vader')

Doc-term matrix shape:  (15593, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (16245, 1000)
Doc-term matrix shape:  (10106, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,help,driver,uber
1,uber,uber,driver
2,please,charged,email
3,driver,ride,app
4,account,get,get
5,get,drivers,ride
6,app,service,account
7,ride,app,still
8,email,trip,sent
9,support,order,need



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,please help,customer service,uber eats
1,need help,uber driver,uber driver
2,phone number,driver cancelled,ride pass
3,uber driver,cancellation fee,promo code
4,customer service,uber eats,still waiting
5,credit card,got charged,left phone
6,would like,last night,sent dm
7,uber account,cancelled trip,uber app
8,uber eats,get charged,email address
9,please check,charged cancellation,last night



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,left phone uber,charged cancellation fee,left phone uber
1,please check dm,driver cancelled trip,email address email
2,please let know,worst customer service,uber last night
3,customer care number,want money back,still waiting response
4,want money back,something went wrong,email id email
5,lost phone uber,lost phone uber,uber eats deliver
6,left phone car,driver cancelled ride,im still waiting
7,friend left phone,charged ride never,mail id email
8,charged cancellation fee,driver never showed,havent heard anything
9,please help get,get money back,left phone car




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,help uber need account phone driver number,driver uber charged ride cancelled trip cancel,email sent dm address account id uber
1,help need section disabled invoices trouble error,charged cancelled trip ride driver cancel fee,sent dm check response uber reply message
2,thanks dm sent check email thank reply,uber account phone driver lost email help,uber driver account app ride eats phone
3,number phone email contact account support cus...,charged account ride order got email help,dm check reply need whats follow send
4,thanks phone number left car contact lost,order wrong food cancelled driver ubereats del...,response waiting reply received hours ive app
5,account email support thanks card credit address,uber cancelled charged order got service eats,account disabled need app ride working delete
6,account uber dm check credit card disabled,ride drivers uber order cancel time eats,uber eats response account disabled deliver dm
7,email uber thank help phone left address,ride driver phone wrong lost service customer,app ride order eats uber working code
8,support uber customer app team issue help,cancelled account ride order hacked pay cancel,waiting reply order im eats hour minutes
9,need uber support contact email asap driver,drivers cancelled phone lost got number car,ride need pass uber reply waiting charge


In [ ]:
topicsByTypes(uber_tweets, 'sentiment_tweet')

Doc-term matrix shape:  (1700, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (23771, 1000)
Doc-term matrix shape:  (16473, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks,driver,uber
1,uber,uber,driver
2,driver,help,please
3,done,get,email
4,thank,ride,help
5,help,app,get
6,get,account,ride
7,please,charged,account
8,ride,drivers,need
9,would,cant,app



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,customer service,customer service,please help
1,would love,uber driver,uber eats
2,ride pass,please help,uber driver
3,please help,driver cancelled,need help
4,thank much,cancellation fee,phone number
5,done thanks,uber eats,email address
6,uber eats,last night,please check
7,promo code,phone number,left phone
8,sent dm,uber account,sent dm
9,email address,got charged,ride pass



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks quick response,charged cancellation fee,left phone uber
1,thanks sent dm,driver cancelled trip,please check dm
2,great customer service,want money back,email id email
3,please respond dm,left phone uber,email address email
4,thanks prompt response,worst customer service,left phone car
5,get phone back,lost phone uber,uber last night
6,would love get,driver never showed,please let know
7,another ride pass,get money back,uber eats deliver
8,promo code received,something went wrong,change phone number
9,thanks charging ride,driver cancelled ride,customer care number




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks sent thank dm help email uber,uber driver help charged ride account trip,email help sent dm uber account need
1,thank email sent help appreciate response got,account help phone uber email number app,help dm sent uber need driver account
2,help uber driver service great good ride,service customer order worst food drivers app,dm sent check reply message respond response
3,sent dm help email response forward asap,uber service customer driver worst drivers phone,help need email dm asap urgent pls
4,help need working app absolutely yes talk,order uber food wrong driver eats ubereats,sent message response note help thank ive
5,service email customer great good support really,uber account charged got drivers ride order,number phone contact account need customer left
6,email helpful ride working address pass got,drivers app ride dont im cancel time,ride charged driver trip need response pass
7,app working support response code helpful uber,drivers account cancel phone disabled trip dont,account need ride card use credit app
8,love hope response good reply ride soon,app trip account driver email support use,response waiting reply app issue support received
9,good driver hope uber response soon need,help trip drivers charged uber cancelled need,need uber asap contact eats drivers speak


In [ ]:
topicsByTypes(uber_tweets,'sentiment_textBlob')

Doc-term matrix shape:  (10015, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (11822, 1000)
Doc-term matrix shape:  (20107, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,uber,driver,uber
1,driver,uber,driver
2,ride,service,help
3,get,get,please
4,help,ride,get
5,thanks,app,email
6,app,help,account
7,account,drivers,ride
8,please,account,app
9,drivers,wrong,charged



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,uber driver,customer service,please help
1,please help,uber driver,uber driver
2,customer service,please help,uber eats
3,first time,uber eats,need help
4,uber eats,account disabled,phone number
5,phone number,cancellation fee,left phone
6,promo code,last night,last night
7,last night,phone number,uber account
8,uber account,driver cancelled,cancellation fee
9,need help,customer support,phone uber



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,left phone uber,worst customer service,left phone uber
1,first time using,want money back,charged cancellation fee
2,charged cancellation fee,charged cancellation fee,lost phone uber
3,able book cab,driver cancelled trip,uber last night
4,charged full price,something went wrong,want money back
5,please let know,terrible customer service,left phone car
6,driver cancelled trip,horrible customer service,driver cancelled trip
7,able use uber,customer service ever,please check dm
8,able book ride,poor customer service,get money back
9,want full refund,worst service ever,customer care number




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,uber thanks driver help ride account app,uber driver service customer ride help wrong,email help uber driver account need phone
1,thanks sent dm email check reply response,service customer worst terrible experience hor...,email address id mail registered mob replied
2,account help email able number phone new,account disabled help email unable support phone,dm sent check reply response thank message
3,ride free able charged code pass promo,order wrong food ubereats delivered missing cold,help need account asap email pls issue
4,uber ride driver account available new phone,uber eats order worst food experience missing,uber phone account number eats contact left
5,uber drivers available free help need really,account disabled driver service order uber worst,sent response message note app thank ive
6,help need really driver able ride trip,drivers time wtf account long worst disabled,phone driver number left contact car lost
7,email drivers really need ride sent uber,ride charged help wtf uber unable wrong,account number phone need contact app support
8,phone drivers number new account really car,worst drivers help wrong experience driver phone,app response waiting order support issue reply
9,drivers account trip able service charged cancel,wrong complaint charged drivers worst email trip,response account waiting driver trip help reply


## Hulu

In [ ]:
hulu_tweets = pd.read_csv('/content/gdrive/MyDrive/ai4se/pp_files/hulu.csv')
hulu_tweets = hulu_tweets.dropna()
hulu_tweets.head()

,Unnamed: 0,text
0,721,wife hate new interface enjoy content getting ...
1,723,hi someone changed password hacked asked reset...
2,733,sounds like generic yourenotgonnafixit replh
3,737,like student discount
4,1403,yes message error playing video iphone


In [ ]:
# vader
hulu_tweets = compute_vader_scores(hulu_tweets)
# text blob
hulu_tweets = compute_textBlob_scores(hulu_tweets)

In [ ]:
# tweet nlp
hulu_tweets = computeTweetSentiment(hulu_tweets)

In [ ]:
hulu_tweets.head()

,Unnamed: 0,Unnamed: 0.1,text,sentiment_vader,sentiment_textBlob,sentiment_tweet
0,0,721,wife hate new interface enjoy content getting ...,-0.6124,-0.315909,-0.931192
1,1,723,hi someone changed password hacked asked reset...,-0.4019,0.000000,-0.770403
2,2,733,sounds like generic yourenotgonnafixit replh,0.3612,0.000000,-0.810058
3,3,737,like student discount,0.3612,0.000000,0.000000
4,4,1403,yes message error playing video iphone,0.2023,0.000000,-0.546047


In [ ]:
hulu_tweets.to_csv('/content/hulu_with_sentiments.csv')

In [ ]:
# load if already saved because already processed.
hulu_tweets = pd.read_csv('/content/hulu_with_sentiments.csv')

In [ ]:
# positive sentiments
getForAll(hulu_tweets,'>0')

Doc-term matrix shape:  (5926, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (6147, 1000)
Doc-term matrix shape:  (1500, 1000)
Comparative Analysis for sentiment>0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,hulu,live,thanks
1,tv,hulu,hulu
2,live,tv,thank
3,please,new,live
4,help,watch,tv
5,watch,app,good
6,thanks,thanks,would
7,like,get,working
8,app,im,great
9,get,available,love



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,live tv,live tv,live tv
1,hulu live,hulu live,hulu live
2,smart tv,smart tv,would love
3,free trial,free trial,love hulu
4,please help,apple tv,apple tv
5,apple tv,new episodes,thank much
6,trying watch,hulu app,free trial
7,please fix,trying watch,fire stick
8,hulu app,able watch,would great
9,need help,watch live,working fine



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,hulu live tv,hulu live tv,hulu live tv
1,samsung smart tv,samsung smart tv,amazon fire stick
2,amazon fire stick,watch live tv,love hulu live
3,live tv service,live tv service,thanks getting back
4,apple tv app,amazon fire stick,episodes good doctor
5,vizio smart tv,watch hulu live,please bring back
6,start free trial,havent able watch,sunday night football
7,amazon fire tv,apple tv app,please come brazil
8,watch live tv,keep getting error,yes live tv
9,episodes good doctor,im trying watch,watch hulu live




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,hulu help tv live thanks watch app,live tv hulu watch app new available,thanks thank working hulu good great help
1,thanks fixed working reply worked ok response,thanks new season episodes available episode f...,thank hulu live fixed good yes got
2,help need account chat email hi spotify,new season available episodes episode hulu shows,yes hulu tv live working love good
3,thank fixed ok season good got working,available season live thanks tv rick morty,yes thanks thank tried figured air content
4,tv live smart help thank samsung thanks,hulu free watch trial able fine help,working fine started app today update finally
5,hulu support free account add spotify season,hulu app smart tv samsung apple tvs,good episodes doctor new working got watch
6,yes app smart season new episodes tv,hulu live new episodes thanks episode buffering,love season new hulu guys episodes like
7,yes hulu free support working trial tried,fine working streaming buffering roku issues s...,great new im service update streaming ok
8,season hulu watch live yes game help,free trial service season month charged got,love good working live shows tv watch
9,season need live free yes like add,season watch fine rick able morty buffering,got fixed finally help app worked love


In [ ]:
# negative sentiments
getForAll(hulu_tweets,'<0')

Doc-term matrix shape:  (4957, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (3801, 1000)
Doc-term matrix shape:  (6911, 1000)
Comparative Analysis for sentiment<0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,hulu,hulu,hulu
1,live,game,live
2,tv,live,watch
3,watch,watch,tv
4,error,tv,app
5,app,app,cant
6,get,im,get
7,im,get,im
8,new,new,error
9,cant,service,service



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,live tv,live tv,live tv
1,hulu live,trying watch,hulu live
2,trying watch,hulu live,trying watch
3,getting error,world series,cant watch
4,keep getting,cant watch,keep getting
5,fire stick,apple tv,hulu app
6,error code,new interface,getting error
7,hulu app,hulu app,every time
8,world series,customer service,apple tv
9,error message,playback failure,world series



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,keep getting error,watch world series,keep getting error
1,getting error message,sec championship game,hulu live tv
2,hulu live tv,hulu live tv,getting error message
3,amazon fire stick,world series game,samsung smart tv
4,im trying watch,get shit together,live tv service
5,getting error code,im trying watch,im trying watch
6,buffer buffer buffer,live tv service,getting error code
7,watch live tv,hate new interface,every time try
8,watch world series,watch live tv,watch live tv
9,amazon fire tv,apple tv app,cant watch anything




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,hulu live tv error watch app getting,game live watch hulu tv app im,hulu live tv watch app error getting
1,error getting code message loading try trying,game live stream buffering watch cubs sec,live tv buffering game service streaming issues
2,live tv buffering game error stream streaming,live tv service hulu buffering streaming apple,error getting live code game message tv
3,watch game im ads trying pay season,new interface game terrible app roku hate,watch im trying game want ads episode
4,hulu account pay charged cancel cancelled comm...,missing live season episodes tv service episode,hulu account help service charged cancelled ca...
5,service cancel account customer streaming fix ...,watch service trying im shit streaming customer,hulu watch app working trying keeps game
6,service ads pay tv commercials im watch,service hulu customer fix game account sucks,game buffering hulu fix stream new issues
7,season tv missing episodes episode error new,hulu live new interface watch hate terrible,new hulu interface live error episodes getting
8,problem ads season fix stick im amazon,im commercials watching ads trouble really com...,ads im commercials fix hulu pay service
9,problem new interface hulu live pay commercials,help im error getting trouble episodes missing,service watch hulu customer error sucks try


In [ ]:
# neutral sentiment
getForAll(hulu_tweets,'==0')

Doc-term matrix shape:  (4937, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (5872, 1000)
Doc-term matrix shape:  (7409, 1000)
Comparative Analysis for sentiment==0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,hulu,hulu,hulu
1,live,app,tv
2,tv,watch,live
3,season,get,watch
4,watch,help,season
5,app,season,get
6,get,tv,app
7,new,error,new
8,cant,please,shows
9,roku,cant,show



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,live tv,apple tv,live tv
1,hulu live,rick morty,hulu live
2,apple tv,hulu app,apple tv
3,rick morty,keep getting,rick morty
4,hulu app,getting error,hulu app
5,new episodes,every time,smart tv
6,trying watch,please help,new episodes
7,xbox one,trying watch,xbox one
8,season rick,xbox one,trying watch
9,cant watch,fire stick,fire stick



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,hulu live tv,season rick morty,hulu live tv
1,season rick morty,keep getting error,season rick morty
2,rick morty season,rick morty season,rick morty season
3,watch live tv,amazon fire stick,amazon fire stick
4,last man standing,getting error message,samsung smart tv
5,law amp order,every time try,watch live tv
6,sunday night football,im trying watch,law amp order
7,im trying watch,getting error code,last man standing
8,amp order svu,buffer buffer buffer,sunday night football
9,rick amp morty,error playing video,amp order svu




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,hulu season live tv watch available rick,hulu season help app watch rick morty,hulu live tv season watch available app
1,season rick morty available new going episodes,season rick morty going coming episodes gonna,season rick morty episodes available new going
2,hulu account isnt spotify app working netflix,help need account season getting error rick,hulu help need account spotify netflix student
3,new episode episodes watch shows app roku,hulu help need account spotify student email,help need watch new episodes account episode
4,working app roku rick morty isnt xbox,working tv app roku apple help isnt,episode new episodes watch shows roku im
5,watch rick morty game trying im app,working error getting message app code xbox,watch live game available commercials trying s...
6,roku app tv apple new episodes update,working watch thank going trying episode need,roku watch rick morty app working im
7,available roku going live stream streaming game,thank roku account yes add app issue,roku available live working streaming help stream
8,morty rick new going episodes live episode,tv thank apple working watch error help,episode help season roku watching isnt im
9,available app tv episode apple morty rick,rick morty roku account going im need,rick morty episode live new going commercials


In [ ]:
topicsByTypes(hulu_tweets, 'sentiment_vader')

Doc-term matrix shape:  (5926, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (4957, 1000)
Doc-term matrix shape:  (4937, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,hulu,hulu,hulu
1,tv,live,live
2,live,tv,tv
3,please,watch,season
4,help,error,watch
5,watch,app,app
6,thanks,get,get
7,like,im,new
8,app,new,cant
9,get,cant,roku



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,live tv,live tv,live tv
1,hulu live,hulu live,hulu live
2,smart tv,trying watch,apple tv
3,free trial,getting error,rick morty
4,please help,keep getting,hulu app
5,apple tv,fire stick,new episodes
6,trying watch,error code,trying watch
7,please fix,hulu app,xbox one
8,hulu app,world series,season rick
9,need help,error message,cant watch



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,hulu live tv,keep getting error,hulu live tv
1,samsung smart tv,getting error message,season rick morty
2,amazon fire stick,hulu live tv,rick morty season
3,live tv service,amazon fire stick,watch live tv
4,apple tv app,im trying watch,last man standing
5,vizio smart tv,getting error code,law amp order
6,start free trial,buffer buffer buffer,sunday night football
7,amazon fire tv,watch live tv,im trying watch
8,watch live tv,watch world series,amp order svu
9,episodes good doctor,amazon fire tv,rick amp morty




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,hulu help tv live thanks watch app,hulu live tv error watch app getting,hulu season live tv watch available rick
1,thanks fixed working reply worked ok response,error getting code message loading try trying,season rick morty available new going episodes
2,help need account chat email hi spotify,live tv buffering game error stream streaming,hulu account isnt spotify app working netflix
3,thank fixed ok season good got working,watch game im ads trying pay season,new episode episodes watch shows app roku
4,tv live smart help thank samsung thanks,hulu account pay charged cancel cancelled comm...,working app roku rick morty isnt xbox
5,hulu support free account add spotify season,service cancel account customer streaming fix ...,watch rick morty game trying im app
6,yes app smart season new episodes tv,service ads pay tv commercials im watch,roku app tv apple new episodes update
7,yes hulu free support working trial tried,season tv missing episodes episode error new,available roku going live stream streaming game
8,season hulu watch live yes game help,problem ads season fix stick im amazon,morty rick new going episodes live episode
9,season need live free yes like add,problem new interface hulu live pay commercials,available app tv episode apple morty rick


In [ ]:
topicsByTypes(hulu_tweets, 'sentiment_tweet')

Doc-term matrix shape:  (1500, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (6911, 1000)
Doc-term matrix shape:  (7409, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks,hulu,hulu
1,hulu,live,tv
2,thank,watch,live
3,live,tv,watch
4,tv,app,season
5,good,cant,get
6,would,get,app
7,working,im,new
8,great,error,shows
9,love,service,show



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,live tv,live tv,live tv
1,hulu live,hulu live,hulu live
2,would love,trying watch,apple tv
3,love hulu,cant watch,rick morty
4,apple tv,keep getting,hulu app
5,thank much,hulu app,smart tv
6,free trial,getting error,new episodes
7,fire stick,every time,xbox one
8,would great,apple tv,trying watch
9,working fine,world series,fire stick



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,hulu live tv,keep getting error,hulu live tv
1,amazon fire stick,hulu live tv,season rick morty
2,love hulu live,getting error message,rick morty season
3,thanks getting back,samsung smart tv,amazon fire stick
4,episodes good doctor,live tv service,samsung smart tv
5,please bring back,im trying watch,watch live tv
6,sunday night football,getting error code,law amp order
7,please come brazil,every time try,last man standing
8,yes live tv,watch live tv,sunday night football
9,watch hulu live,cant watch anything,amp order svu




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks thank working hulu good great help,hulu live tv watch app error getting,hulu live tv season watch available app
1,thank hulu live fixed good yes got,live tv buffering game service streaming issues,season rick morty episodes available new going
2,yes hulu tv live working love good,error getting live code game message tv,hulu help need account spotify netflix student
3,yes thanks thank tried figured air content,watch im trying game want ads episode,help need watch new episodes account episode
4,working fine started app today update finally,hulu account help service charged cancelled ca...,episode new episodes watch shows roku im
5,good episodes doctor new working got watch,hulu watch app working trying keeps game,watch live game available commercials trying s...
6,love season new hulu guys episodes like,game buffering hulu fix stream new issues,roku watch rick morty app working im
7,great new im service update streaming ok,new hulu interface live error episodes getting,roku available live working streaming help stream
8,love good working live shows tv watch,ads im commercials fix hulu pay service,episode help season roku watching isnt im
9,got fixed finally help app worked love,service watch hulu customer error sucks try,rick morty episode live new going commercials


In [ ]:
topicsByTypes(hulu_tweets, 'sentiment_textBlob')

Doc-term matrix shape:  (6147, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (3801, 1000)
Doc-term matrix shape:  (5872, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,live,hulu,hulu
1,hulu,game,app
2,tv,live,watch
3,new,watch,get
4,watch,tv,help
5,app,app,season
6,thanks,im,tv
7,get,get,error
8,im,new,please
9,available,service,cant



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,live tv,live tv,apple tv
1,hulu live,trying watch,rick morty
2,smart tv,hulu live,hulu app
3,free trial,world series,keep getting
4,apple tv,cant watch,getting error
5,new episodes,apple tv,every time
6,hulu app,new interface,please help
7,trying watch,hulu app,trying watch
8,able watch,customer service,xbox one
9,watch live,playback failure,fire stick



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,hulu live tv,watch world series,season rick morty
1,samsung smart tv,sec championship game,keep getting error
2,watch live tv,hulu live tv,rick morty season
3,live tv service,world series game,amazon fire stick
4,amazon fire stick,get shit together,getting error message
5,watch hulu live,im trying watch,every time try
6,havent able watch,live tv service,im trying watch
7,apple tv app,hate new interface,getting error code
8,keep getting error,watch live tv,buffer buffer buffer
9,im trying watch,apple tv app,error playing video




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,live tv hulu watch app new available,game live watch hulu tv app im,hulu season help app watch rick morty
1,thanks new season episodes available episode f...,game live stream buffering watch cubs sec,season rick morty going coming episodes gonna
2,new season available episodes episode hulu shows,live tv service hulu buffering streaming apple,help need account season getting error rick
3,available season live thanks tv rick morty,new interface game terrible app roku hate,hulu help need account spotify student email
4,hulu free watch trial able fine help,missing live season episodes tv service episode,working tv app roku apple help isnt
5,hulu app smart tv samsung apple tvs,watch service trying im shit streaming customer,working error getting message app code xbox
6,hulu live new episodes thanks episode buffering,service hulu customer fix game account sucks,working watch thank going trying episode need
7,fine working streaming buffering roku issues s...,hulu live new interface watch hate terrible,thank roku account yes add app issue
8,free trial service season month charged got,im commercials watching ads trouble really com...,tv thank apple working watch error help
9,season watch fine rick able morty buffering,help im error getting trouble episodes missing,rick morty roku account going im need


## Spectrum

In [ ]:
spectrum_tweets = pd.read_csv('/content/gdrive/MyDrive/ai4se/pp_files/spectrum.csv')
spectrum_tweets = spectrum_tweets.dropna()
spectrum_tweets.head()

,Unnamed: 0,text
0,16,would like email copy one since spectrum updat...
1,18,received corporate office would like copy
2,21,incorrect information form front faxed overmay...
3,23,actually thats broken link sent incorrect info...
4,27,picture pretty much every day pay per month


In [ ]:
# vader
spectrum_tweets = compute_vader_scores(spectrum_tweets)
# text blob
spectrum_tweets = compute_textBlob_scores(spectrum_tweets)

In [ ]:
# tweet nlp
spectrum_tweets = computeTweetSentiment(spectrum_tweets)

In [ ]:
spectrum_tweets.head()

,Unnamed: 0,text,sentiment_vader,sentiment_textBlob,sentiment_tweet
0,16,would like email copy one since spectrum updat...,0.3612,0.000,0.000000
1,18,received corporate office would like copy,0.3612,0.000,0.000000
2,21,incorrect information form front faxed overmay...,0.0000,0.000,-0.503091
3,23,actually thats broken link sent incorrect info...,-0.4767,-0.200,-0.876917
4,27,picture pretty much every day pay per month,0.4215,0.225,0.000000


In [ ]:
spectrum_tweets.to_csv('/content/spectrum_with_sentiments.csv')

In [ ]:
# load if already saved because already processed.
spectrum_tweets = pd.read_csv('/content/spectrum_with_sentiments.csv')

In [ ]:
# positive sentiments
getForAll(spectrum_tweets, '>0')

Doc-term matrix shape:  (6304, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (5379, 1000)
Doc-term matrix shape:  (1698, 1000)
Comparative Analysis for sentiment>0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,internet,internet,thanks
1,service,service,internet
2,thanks,thanks,thank
3,like,get,service
4,spectrum,spectrum,back
5,get,outage,good
6,please,new,working
7,help,time,would
8,time,im,great
9,im,good,love



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,customer service,customer service,came back
1,internet service,internet service,customer service
2,looks like,time warner,internet working
3,please fix,internet outage,internet back
4,time warner,last night,internet service
5,internet outage,came back,back thanks
6,please help,new york,would nice
7,service restored,im paying,would love
8,would like,cable box,service restored
9,last night,whats going,cable box



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,time warner cable,time warner cable,came back thanks
1,drop comedy central,check result fast,service came back
2,comedy central nickelodeon,result fast internet,thanks quick response
3,central nickelodeon bet,came back thanks,finally came back
4,nickelodeon bet call,thanks quick response,came back minutes
5,bet call go,would love know,sure would nice
6,call go keepviacom,spectrum tv app,would love able
7,san antonio tx,much longer outage,came back hour
8,spectrum tv app,service came back,would love watch
9,would love know,second time week,thanks great service




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,thanks internet service thank help spectrum like,thanks internet service outage spectrum fixed ...,thank thanks yes working internet good service
1,thank yes issue resolved responding working reply,thanks ok response came resolved reply quick,thanks internet working service ok good came
2,thanks thank ok response came reply quick,fixed outage internet right area thanks going,yes great sent come properly finally states
3,yes service time channel number dm spectrum,fixed service thank problem customer good ok,internet working service love spectrum good great
4,help need yes internet thanks wifi issue,outage right service new area customer york,working thanks yes app website fine response
5,service help customer restored need spectrum p...,service internet customer better day great days,service working restored customer great came got
6,like spectrum im channels looks watch tv,good service outage thank ok im thing,good working ok im thing know looks
7,outage like area looks tx help restored,new outage york good area spectrum modem,ok love great fixed help cool watch
8,outage spectrum good tx area resolved time,ok thank internet service love working great,ok restored service internet good queens area
9,good im ok like help thing looks,new right fixed internet im modem customer,fixed finally came got response time hour


In [ ]:
# negative sentiments
getForAll(spectrum_tweets, '<0')

Doc-term matrix shape:  (6454, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (5421, 1000)
Doc-term matrix shape:  (9833, 1000)
Comparative Analysis for sentiment<0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,internet,internet,internet
1,service,service,service
2,spectrum,spectrum,spectrum
3,get,get,get
4,pay,time,outage
5,time,im,time
6,cable,cable,im
7,im,game,cable
8,going,worst,cant
9,worst,customer,fix



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,customer service,customer service,customer service
1,time warner,internet service,internet outage
2,internet service,time warner,internet service
3,shit together,shit together,time warner
4,get shit,get shit,cant even
5,cant even,worst internet,cant get
6,worst internet,internet slow,whats going
7,internet connection,cant even,internet connection
8,im getting,cable box,please fix
9,cable box,service sucks,cable internet



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,get shit together,get shit together,get shit together
1,worst customer service,worst customer service,worst customer service
2,time warner cable,time warner cable,time warner cable
3,cant even watch,horrible customer service,cant even watch
4,worst service ever,worst service ever,internet isnt working
5,horrible customer service,worst internet provider,horrible customer service
6,worst internet provider,every single day,spectrum tv app
7,poor customer service,cant even watch,another internet outage
8,worst company ever,worst cable company,another day another
9,pay way much,worst company ever,worst service ever




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,internet service spectrum pay worst cable time,internet service spectrum worst sucks time slow,internet outage service spectrum cable fix time
1,service customer worst horrible terrible poor ...,service worst customer horrible terrible compa...,outage internet area zip ny slow manhattan
2,internet worst service provider bad customer t...,game watch cable shit spectrum channel im,outage service customer worst horrible area long
3,spectrum worst cable sucks company tv time,worst shit fucking fuck company wifi fix,spectrum outage cable wifi tv time im
4,suck guys shit fucking yall fuck wifi,worst game spectrum cable provider watch company,spectrum worst internet sucks cable bad tv
5,shit worst fix fuck fucking company wifi,sucks spectrum wifi fucking bad internet ass,fix shit fucking worst wifi suck guys
6,pay worst suck cable month provider guys,game worst sucks internet watch service fucking,cable time worst warner wifi suck company
7,sucks wifi pay fucking spectrum ass fuck,spectrum bad shit terrible game internet service,wifi sucks suck fucking im ass trash
8,spectrum shit pay service bad terrible fix,fucking fuck terrible suck game service trash,suck guys getting yall im bad channels
9,time warner cable sucks bad wifi worse,bad time internet fucking cable outage warner,suck time issues working guys wifi warner


In [ ]:
# neutral sentiment
getForAll(spectrum_tweets,'==0')

Doc-term matrix shape:  (6723, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (8681, 1000)
Doc-term matrix shape:  (7950, 1000)
Comparative Analysis for sentiment==0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,internet,internet,internet
1,outage,service,outage
2,service,outage,service
3,spectrum,spectrum,spectrum
4,area,get,area
5,cable,cable,cable
6,get,area,get
7,going,back,time
8,back,going,back
9,time,time,going



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,internet outage,internet outage,internet service
1,internet service,internet service,zip code
2,zip code,zip code,time warner
3,whats going,cable internet,whats going
4,time warner,whats going,cable internet
5,cable internet,time warner,outage area
6,outage area,internet cable,internet outage
7,internet working,customer service,cable box
8,cable box,outage area,last night
9,service outage,last night,customer service



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,time warner cable,time warner cable,time warner cable
1,outage zip code,spectrum tv app,spectrum tv app
2,spectrum tv app,since last night,outage zip code
3,since last night,outage zip code,since last night
4,another day another,internet keeps going,comedy central nickelodeon
5,whats going internet,another day another,drop comedy central
6,cable internet phone,drop comedy central,central nickelodeon bet
7,internet keeps going,comedy central nickelodeon,nickelodeon bet call
8,outage los angeles,central nickelodeon bet,bet call go
9,internet isnt working,nickelodeon bet call,call go keepviacom




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,internet outage area service going cable working,internet outage service area cable working going,internet outage service area cable going spectrum
1,outage area service zip code nyc brooklyn,outage area service zip code brooklyn update,outage area service zip code eta nyc
2,service going spectrum cable time issues whats,service spectrum going cable time thank customer,service spectrum time going cable tv channel
3,working spectrum cable area tv going app,spectrum working thank cable tv app wifi,spectrum cable tv working time app channel
4,working service isnt internet outage fl ca,thank yes service internet working resolved re...,area going whats working cable code zip
5,area code issues outages zip nc known,working service isnt area demand code ca,area spectrum code zip outages tv service
6,going whats working hours wifi fixed keeps,area going whats cable code outages issues,cable time warner box working code service
7,cable time box area code warner zip,area spectrum issues tv code outages app,working tv channels app watch channel service
8,zip code outages tv time services im,cable area time box warner service phone,dm sent help time need im account
9,wifi fix time im issues day work,fix help yes wifi need dont guys,time warner working zip pay code month


In [ ]:
topicsByTypes(spectrum_tweets, 'sentiment_vader')

Doc-term matrix shape:  (6304, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (6454, 1000)
Doc-term matrix shape:  (6723, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,internet,internet,internet
1,service,service,outage
2,thanks,spectrum,service
3,like,get,spectrum
4,spectrum,pay,area
5,get,time,cable
6,please,cable,get
7,help,im,going
8,time,going,back
9,im,worst,time



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,customer service,customer service,internet outage
1,internet service,time warner,internet service
2,looks like,internet service,zip code
3,please fix,shit together,whats going
4,time warner,get shit,time warner
5,internet outage,cant even,cable internet
6,please help,worst internet,outage area
7,service restored,internet connection,internet working
8,would like,im getting,cable box
9,last night,cable box,service outage



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,time warner cable,get shit together,time warner cable
1,drop comedy central,worst customer service,outage zip code
2,comedy central nickelodeon,time warner cable,spectrum tv app
3,central nickelodeon bet,cant even watch,since last night
4,nickelodeon bet call,worst service ever,another day another
5,bet call go,horrible customer service,whats going internet
6,call go keepviacom,worst internet provider,cable internet phone
7,san antonio tx,poor customer service,internet keeps going
8,spectrum tv app,worst company ever,outage los angeles
9,would love know,pay way much,internet isnt working




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks internet service thank help spectrum like,internet service spectrum pay worst cable time,internet outage area service going cable working
1,thank yes issue resolved responding working reply,service customer worst horrible terrible poor ...,outage area service zip code nyc brooklyn
2,thanks thank ok response came reply quick,internet worst service provider bad customer t...,service going spectrum cable time issues whats
3,yes service time channel number dm spectrum,spectrum worst cable sucks company tv time,working spectrum cable area tv going app
4,help need yes internet thanks wifi issue,suck guys shit fucking yall fuck wifi,working service isnt internet outage fl ca
5,service help customer restored need spectrum p...,shit worst fix fuck fucking company wifi,area code issues outages zip nc known
6,like spectrum im channels looks watch tv,pay worst suck cable month provider guys,going whats working hours wifi fixed keeps
7,outage like area looks tx help restored,sucks wifi pay fucking spectrum ass fuck,cable time box area code warner zip
8,outage spectrum good tx area resolved time,spectrum shit pay service bad terrible fix,zip code outages tv time services im
9,good im ok like help thing looks,time warner cable sucks bad wifi worse,wifi fix time im issues day work


In [ ]:
topicsByTypes(spectrum_tweets, 'sentiment_tweet')

Doc-term matrix shape:  (1698, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (9833, 1000)
Doc-term matrix shape:  (7950, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks,internet,internet
1,internet,service,outage
2,thank,spectrum,service
3,service,get,spectrum
4,back,outage,area
5,good,time,cable
6,working,im,get
7,would,cable,time
8,great,cant,back
9,love,fix,going



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,came back,customer service,internet service
1,customer service,internet outage,zip code
2,internet working,internet service,time warner
3,internet back,time warner,whats going
4,internet service,cant even,cable internet
5,back thanks,cant get,outage area
6,would nice,whats going,internet outage
7,would love,internet connection,cable box
8,service restored,please fix,last night
9,cable box,cable internet,customer service



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,came back thanks,get shit together,time warner cable
1,service came back,worst customer service,spectrum tv app
2,thanks quick response,time warner cable,outage zip code
3,finally came back,cant even watch,since last night
4,came back minutes,internet isnt working,comedy central nickelodeon
5,sure would nice,horrible customer service,drop comedy central
6,would love able,spectrum tv app,central nickelodeon bet
7,came back hour,another internet outage,nickelodeon bet call
8,would love watch,another day another,bet call go
9,thanks great service,worst service ever,call go keepviacom




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thank thanks yes working internet good service,internet outage service spectrum cable fix time,internet outage service area cable going spectrum
1,thanks internet working service ok good came,outage internet area zip ny slow manhattan,outage area service zip code eta nyc
2,yes great sent come properly finally states,outage service customer worst horrible area long,service spectrum time going cable tv channel
3,internet working service love spectrum good great,spectrum outage cable wifi tv time im,spectrum cable tv working time app channel
4,working thanks yes app website fine response,spectrum worst internet sucks cable bad tv,area going whats working cable code zip
5,service working restored customer great came got,fix shit fucking worst wifi suck guys,area spectrum code zip outages tv service
6,good working ok im thing know looks,cable time worst warner wifi suck company,cable time warner box working code service
7,ok love great fixed help cool watch,wifi sucks suck fucking im ass trash,working tv channels app watch channel service
8,ok restored service internet good queens area,suck guys getting yall im bad channels,dm sent help time need im account
9,fixed finally came got response time hour,suck time issues working guys wifi warner,time warner working zip pay code month


In [ ]:
topicsByTypes(spectrum_tweets, 'sentiment_textBlob')

Doc-term matrix shape:  (5379, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (5421, 1000)
Doc-term matrix shape:  (8681, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,internet,internet,internet
1,service,service,service
2,thanks,spectrum,outage
3,get,get,spectrum
4,spectrum,time,get
5,outage,im,cable
6,new,cable,area
7,time,game,back
8,im,worst,going
9,good,customer,time



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,customer service,customer service,internet outage
1,internet service,internet service,internet service
2,time warner,time warner,zip code
3,internet outage,shit together,cable internet
4,last night,get shit,whats going
5,came back,worst internet,time warner
6,new york,internet slow,internet cable
7,im paying,cant even,customer service
8,cable box,cable box,outage area
9,whats going,service sucks,last night



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,time warner cable,get shit together,time warner cable
1,check result fast,worst customer service,spectrum tv app
2,result fast internet,time warner cable,since last night
3,came back thanks,horrible customer service,outage zip code
4,thanks quick response,worst service ever,internet keeps going
5,would love know,worst internet provider,another day another
6,spectrum tv app,every single day,drop comedy central
7,much longer outage,cant even watch,comedy central nickelodeon
8,service came back,worst cable company,central nickelodeon bet
9,second time week,worst company ever,nickelodeon bet call




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks internet service outage spectrum fixed ...,internet service spectrum worst sucks time slow,internet outage service area cable working going
1,thanks ok response came resolved reply quick,service worst customer horrible terrible compa...,outage area service zip code brooklyn update
2,fixed outage internet right area thanks going,game watch cable shit spectrum channel im,service spectrum going cable time thank customer
3,fixed service thank problem customer good ok,worst shit fucking fuck company wifi fix,spectrum working thank cable tv app wifi
4,outage right service new area customer york,worst game spectrum cable provider watch company,thank yes service internet working resolved re...
5,service internet customer better day great days,sucks spectrum wifi fucking bad internet ass,working service isnt area demand code ca
6,good service outage thank ok im thing,game worst sucks internet watch service fucking,area going whats cable code outages issues
7,new outage york good area spectrum modem,spectrum bad shit terrible game internet service,area spectrum issues tv code outages app
8,ok thank internet service love working great,fucking fuck terrible suck game service trash,cable area time box warner service phone
9,new right fixed internet im modem customer,bad time internet fucking cable outage warner,fix help yes wifi need dont guys


## Amazon

In [ ]:
amazon_tweets = pd.read_csv('/content/gdrive/MyDrive/ai4se/pp_files/amazon.csv')
amazon_tweets = amazon_tweets.dropna()
amazon_tweets.head()

,Unnamed: 0,text
0,322,different people given different answers stil...
1,325,way drop ball customer service pissed right
2,327,want amazon payments account closed dm please
3,331,also beim addams familyfilm prime sind bild un...
4,342,thanks style advice look think halloween2017 f...


In [ ]:
# vader
amazon_tweets = compute_vader_scores(amazon_tweets)
# text blob
amazon_tweets = compute_textBlob_scores(amazon_tweets)

In [ ]:
# tweet nlp
amazon_tweets = computeTweetSentiment(amazon_tweets)

In [ ]:
amazon_tweets.head()

,Unnamed: 0,text,sentiment_vader,sentiment_textBlob,sentiment_tweet
0,322,different people given different answers stil...,0.0000,0.000000,-0.597357
1,325,way drop ball customer service pissed right,-0.7430,0.285714,-0.828041
2,327,want amazon payments account closed dm please,0.5106,-0.100000,-0.594373
3,331,also beim addams familyfilm prime sind bild un...,0.0000,0.000000,0.000000
4,342,thanks style advice look think halloween2017 f...,0.4404,0.200000,0.566124


In [ ]:
amazon_tweets.to_csv('/content/amazon_with_sentiments.csv')

In [ ]:
# load if already saved because already processed.
amazon_tweets = pd.read_csv('/content/amazon_with_sentiments.csv')

In [ ]:
# positive sentiments
getForAll(amazon_tweets, '>0')

Doc-term matrix shape:  (43033, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (26226, 1000)
Doc-term matrix shape:  (9005, 1000)
Comparative Analysis for sentiment>0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,amazon,amazon,thanks
1,order,delivery,amazon
2,please,order,thank
3,delivery,thanks,delivery
4,help,prime,yes
5,prime,get,prime
6,customer,time,service
7,get,customer,today
8,thanks,service,order
9,delivered,one,customer



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,customer service,customer service,customer service
1,customer care,amazon prime,amazon prime
2,amazon prime,many times,thank much
3,please help,day delivery,day delivery
4,day delivery,customer care,delivered today
5,amazon pay,first time,thanks help
6,next day,next day,day shipping
7,prime membership,day shipping,thanks quick
8,gift card,prime membership,muchas gracias
9,delivery date,filled form,thanks amazon



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,next day delivery,next day delivery,great customer service
1,amazon pay balance,one day delivery,next day delivery
2,one day delivery,amazon pay balance,best customer service
3,please let know,two day shipping,thanks quick response
4,two day shipping,great customer service,thanks quick reply
5,amazon customer care,thanks quick response,amazing customer service
6,amazon prime video,please make sure,thanks prompt response
7,amazon customer service,one day shipping,great indian festival
8,customer care executive,already filled form,keep good work
9,called customer care,social media team,one day delivery




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,amazon help order thanks delivery prime customer,thanks amazon order delivery help prime customer,yes thanks thank help amazon today got
1,yes thanks help thank order need email,thanks sorted prompt resolved worked lot ruining,thanks thank help amazon ok got great
2,yes amazon prime logistics delivery pay times,delivery day prime shipping free today delivered,thank ok help amazon service great customer
3,thanks amazon yes logistics por prime sold,ok delivery thank order day delivered filled,amazon delivery service customer prime deliver...
4,thank thanks amazon ok great response reply,amazon ok prime available india thank free,service customer great help best amazing helpful
5,help amazon yes thank need thanks account,service customer delivery day prime great good,help delivered today got order appreciate hope
6,customer service care support issue email contact,filled form prime day details delivery times,amazon help fulfilled india company prime great
7,order number details amazon refund check shared,available prime help free shipping customer able,delivery day love prime great shipping date
8,email want account details dont shared prime,order filled prime amazon form service customer,gracias ok muchas por la perfecto acuerdo
9,prime order membership customer shipping servi...,available time ok product india customer service,great ok love got good guys job


In [ ]:
# negative sentiments
getForAll(amazon_tweets, '<0')

Doc-term matrix shape:  (29011, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (24035, 1000)
Doc-term matrix shape:  (43299, 1000)
Comparative Analysis for sentiment<0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,amazon,amazon,amazon
1,delivery,delivery,order
2,order,service,delivery
3,service,order,service
4,prime,customer,customer
5,customer,prime,prime
6,get,delivered,get
7,delivered,time,delivered
8,time,get,time
9,product,days,dont



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,customer service,customer service,customer service
1,day delivery,customer care,customer care
2,amazon prime,amazon prime,day delivery
3,next day,day delivery,amazon prime
4,prime membership,worst service,next day
5,cancel order,pathetic service,prime membership
6,pathetic service,next day,please help
7,worst service,prime membership,day shipping
8,day shipping,delivery date,delivery date
9,customer care,worst customer,cancel order



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,next day delivery,worst customer service,next day delivery
1,worst customer service,next day delivery,worst customer service
2,one day delivery,worst service ever,one day delivery
3,amazon pay balance,poor customer service,amazon pay balance
4,worst service ever,amazon pay balance,two day shipping
5,fire tv stick,one day delivery,still havent received
6,poor customer service,expected delivery date,worst service ever
7,two day shipping,pathetic customer service,poor customer service
8,delivered wrong address,amazon customer service,amazon customer service
9,customer service ever,delivered wrong address,get money back




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,amazon order service delivery customer prime d...,service amazon delivery order customer prime w...,amazon order delivery service customer prime d...
1,service customer worst pathetic poor experienc...,service customer worst pathetic care poor expe...,service customer worst care pathetic poor expe...
2,delivery day prime package shipping today pay,amazon product order wrong help refund fake,delivery day prime service package shipping today
3,amazon account prime india dont pay product,order delivered wrong received product help ca...,amazon account prime dont india pay money
4,prime order amazon cancel pay cancelled day,order amazon delivery worst pathetic experienc...,order amazon prime cancel cancelled service worst
5,delivery amazon order product worst pathetic i...,prime order days late shipping customer member...,prime account help days dont shipping membership
6,delivered package prime wrong ordered today se...,delivered worst prime package product wrong or...,delivery dont help account email order want
7,product prime refund pay worst return day,worst time product experience refund dont days,product dont refund want time money prime
8,time dont package deliver want really waste,worst package help im today experience order,dont package time delivered know order want
9,problem time product delivered prime worst acc...,time delivered deliver package service waste dont,help dont customer care want day need


In [ ]:
# neutral sentiment
getForAll(amazon_tweets, '==0')

Doc-term matrix shape:  (29705, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (51488, 1000)
Doc-term matrix shape:  (49445, 1000)
Comparative Analysis for sentiment==0 Tweets

-- Unigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,order,amazon,amazon
1,delivery,order,order
2,delivered,delivery,delivery
3,prime,prime,prime
4,still,delivered,please
5,today,get,delivered
6,get,please,get
7,email,still,email
8,day,help,today
9,one,email,one



-- Bigrams -- 


,Vader,TextBlob,TweetNLP Roberta
0,day delivery,customer service,amazon prime
1,customer service,amazon prime,day delivery
2,delivery date,day delivery,delivery date
3,next day,please help,next day
4,amzl us,customer care,customer service
5,day shipping,next day,amazon pay
6,still waiting,delivery date,please help
7,delivered today,day shipping,prime membership
8,havent received,prime membership,customer care
9,one day,amazon pay,day shipping



-- Trigrams --


,Vader,TextBlob,TweetNLP Roberta
0,next day delivery,next day delivery,next day delivery
1,one day delivery,amazon pay balance,amazon pay balance
2,two day shipping,one day delivery,one day delivery
3,carrier amzl us,two day shipping,please let know
4,still havent received,supposed delivered today,two day shipping
5,supposed delivered today,get money back,carrier amzl us
6,one day shipping,please let know,prime next day
7,prime next day,carrier amzl us,fire tv stick
8,shipped amzl us,still havent received,one day shipping
9,whats point prime,one day shipping,amazon prime video




-- Topics --


,Vader,TextBlob,TweetNLP Roberta
0,order delivery delivered today prime day received,amazon order delivery delivered prime help today,amazon order delivery prime delivered help today
1,order placed id details amazonin status place,amazon prime logistics por account pay india,amazon prime por logistics pay india sold
2,email received sent account got details havent,yes email times reply today yesterday delivery,order amazon help number details need id
3,delivered today package says received supposed...,delivery day prime today delivered package says,email help sent need details account received
4,delivery email today says date order amzl,help email need thank account sent received,help need delivery prime day hi hey
5,amzl says email prime carrier sent shipped,email sent received account got details reply,delivered today package says amazon received y...
6,amzl waiting details time response update refund,thank order amazon got package resolved arrived,details shared link time contact check provided
7,received amzl prime havent carrier says shipped,delivered package today says received product ...,delivery amazon date email help details order
8,gracias que la el en por es,delivery email amazon help order date today,que el en la por es lo
9,today details received sent ordered link prime,customer service refund product details delive...,details prime says today help shared delivered


In [ ]:
topicsByTypes(amazon_tweets, 'sentiment_vader')

Doc-term matrix shape:  (43033, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (29011, 1000)
Doc-term matrix shape:  (29705, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,amazon,amazon,order
1,order,delivery,delivery
2,please,order,delivered
3,delivery,service,prime
4,help,prime,still
5,prime,customer,today
6,customer,get,get
7,get,delivered,email
8,thanks,time,day
9,delivered,product,one



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,customer service,customer service,day delivery
1,customer care,day delivery,customer service
2,amazon prime,amazon prime,delivery date
3,please help,next day,next day
4,day delivery,prime membership,amzl us
5,amazon pay,cancel order,day shipping
6,next day,pathetic service,still waiting
7,prime membership,worst service,delivered today
8,gift card,day shipping,havent received
9,delivery date,customer care,one day



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,next day delivery,next day delivery,next day delivery
1,amazon pay balance,worst customer service,one day delivery
2,one day delivery,one day delivery,two day shipping
3,please let know,amazon pay balance,carrier amzl us
4,two day shipping,worst service ever,still havent received
5,amazon customer care,fire tv stick,supposed delivered today
6,amazon prime video,poor customer service,one day shipping
7,amazon customer service,two day shipping,prime next day
8,customer care executive,delivered wrong address,shipped amzl us
9,called customer care,customer service ever,whats point prime




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,amazon help order thanks delivery prime customer,amazon order service delivery customer prime d...,order delivery delivered today prime day received
1,yes thanks help thank order need email,service customer worst pathetic poor experienc...,order placed id details amazonin status place
2,yes amazon prime logistics delivery pay times,delivery day prime package shipping today pay,email received sent account got details havent
3,thanks amazon yes logistics por prime sold,amazon account prime india dont pay product,delivered today package says received supposed...
4,thank thanks amazon ok great response reply,prime order amazon cancel pay cancelled day,delivery email today says date order amzl
5,help amazon yes thank need thanks account,delivery amazon order product worst pathetic i...,amzl says email prime carrier sent shipped
6,customer service care support issue email contact,delivered package prime wrong ordered today se...,amzl waiting details time response update refund
7,order number details amazon refund check shared,product prime refund pay worst return day,received amzl prime havent carrier says shipped
8,email want account details dont shared prime,time dont package deliver want really waste,gracias que la el en por es
9,prime order membership customer shipping servi...,problem time product delivered prime worst acc...,today details received sent ordered link prime


In [ ]:
topicsByTypes(amazon_tweets, 'sentiment_tweet')

Doc-term matrix shape:  (9005, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (43299, 1000)
Doc-term matrix shape:  (49445, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks,amazon,amazon
1,amazon,order,order
2,thank,delivery,delivery
3,delivery,service,prime
4,yes,customer,please
5,prime,prime,delivered
6,service,get,get
7,today,delivered,email
8,order,time,today
9,customer,dont,one



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,customer service,customer service,amazon prime
1,amazon prime,customer care,day delivery
2,thank much,day delivery,delivery date
3,day delivery,amazon prime,next day
4,delivered today,next day,customer service
5,thanks help,prime membership,amazon pay
6,day shipping,please help,please help
7,thanks quick,day shipping,prime membership
8,muchas gracias,delivery date,customer care
9,thanks amazon,cancel order,day shipping



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,great customer service,next day delivery,next day delivery
1,next day delivery,worst customer service,amazon pay balance
2,best customer service,one day delivery,one day delivery
3,thanks quick response,amazon pay balance,please let know
4,thanks quick reply,two day shipping,two day shipping
5,amazing customer service,still havent received,carrier amzl us
6,thanks prompt response,worst service ever,prime next day
7,great indian festival,poor customer service,fire tv stick
8,keep good work,amazon customer service,one day shipping
9,one day delivery,get money back,amazon prime video




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,yes thanks thank help amazon today got,amazon order delivery service customer prime d...,amazon order delivery prime delivered help today
1,thanks thank help amazon ok got great,service customer worst care pathetic poor expe...,amazon prime por logistics pay india sold
2,thank ok help amazon service great customer,delivery day prime service package shipping today,order amazon help number details need id
3,amazon delivery service customer prime deliver...,amazon account prime dont india pay money,email help sent need details account received
4,service customer great help best amazing helpful,order amazon prime cancel cancelled service worst,help need delivery prime day hi hey
5,help delivered today got order appreciate hope,prime account help days dont shipping membership,delivered today package says amazon received y...
6,amazon help fulfilled india company prime great,delivery dont help account email order want,details shared link time contact check provided
7,delivery day love prime great shipping date,product dont refund want time money prime,delivery amazon date email help details order
8,gracias ok muchas por la perfecto acuerdo,dont package time delivered know order want,que el en la por es lo
9,great ok love got good guys job,help dont customer care want day need,details prime says today help shared delivered


In [ ]:
topicsByTypes(amazon_tweets, 'sentiment_textBlob')

Doc-term matrix shape:  (26226, 1000)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Doc-term matrix shape:  (24035, 1000)
Doc-term matrix shape:  (51488, 1000)
Top Unigrams for all 3 types of tweets: 

-- Unigrams -- 


,Postive Tweets,Negative Tweets,Neutral Tweet
0,amazon,amazon,amazon
1,delivery,delivery,order
2,order,service,delivery
3,thanks,order,prime
4,prime,customer,delivered
5,get,prime,get
6,time,delivered,please
7,customer,time,still
8,service,get,help
9,one,days,email



-- Bigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,customer service,customer service,customer service
1,amazon prime,customer care,amazon prime
2,many times,amazon prime,day delivery
3,day delivery,day delivery,please help
4,customer care,worst service,customer care
5,first time,pathetic service,next day
6,next day,next day,delivery date
7,day shipping,prime membership,day shipping
8,prime membership,delivery date,prime membership
9,filled form,worst customer,amazon pay



-- Trigrams --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,next day delivery,worst customer service,next day delivery
1,one day delivery,next day delivery,amazon pay balance
2,amazon pay balance,worst service ever,one day delivery
3,two day shipping,poor customer service,two day shipping
4,great customer service,amazon pay balance,supposed delivered today
5,thanks quick response,one day delivery,get money back
6,please make sure,expected delivery date,please let know
7,one day shipping,pathetic customer service,carrier amzl us
8,already filled form,amazon customer service,still havent received
9,social media team,delivered wrong address,one day shipping




-- Topics --


,Postive Tweets,Negative Tweets,Neutral Tweet
0,thanks amazon order delivery help prime customer,service amazon delivery order customer prime w...,amazon order delivery delivered prime help today
1,thanks sorted prompt resolved worked lot ruining,service customer worst pathetic care poor expe...,amazon prime logistics por account pay india
2,delivery day prime shipping free today delivered,amazon product order wrong help refund fake,yes email times reply today yesterday delivery
3,ok delivery thank order day delivered filled,order delivered wrong received product help ca...,delivery day prime today delivered package says
4,amazon ok prime available india thank free,order amazon delivery worst pathetic experienc...,help email need thank account sent received
5,service customer delivery day prime great good,prime order days late shipping customer member...,email sent received account got details reply
6,filled form prime day details delivery times,delivered worst prime package product wrong or...,thank order amazon got package resolved arrived
7,available prime help free shipping customer able,worst time product experience refund dont days,delivered package today says received product ...
8,order filled prime amazon form service customer,worst package help im today experience order,delivery email amazon help order date today
9,available time ok product india customer service,time delivered deliver package service waste dont,customer service refund product details delive...
